# Nobel Laureates and Discoveries Analysis

This notebook loads data about Nobel laureates and their discoveries into an SQLite database and performs several analyses based on predefined questions.

In [1]:
import sqlite3
import pandas as pd

# Connect to an in-memory SQLite database (or replace ':memory:' with 'nobel_data.db' for a file)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Enable foreign key support
cursor.execute("PRAGMA foreign_keys = ON;")

print("Database connection established and foreign keys enabled.")

Database connection established and foreign keys enabled.


## Database Schema Setup

First, we drop any existing tables to ensure a clean slate. Then, we create the `nobel_laureates` and `discoveries` tables with SQLite-compatible syntax.

In [2]:
# Drop tables if they exist (SQLite compatible)
cursor.execute("DROP TABLE IF EXISTS discoveries;")
cursor.execute("DROP TABLE IF EXISTS nobel_laureates;")

# Create nobel_laureates table (SQLite compatible)
cursor.execute("""
CREATE TABLE nobel_laureates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    field TEXT NOT NULL,
    year_award INTEGER NOT NULL,
    prize_amount REAL NOT NULL,
    age_at_award INTEGER NOT NULL
);
""")

# Create discoveries table (SQLite compatible)
cursor.execute("""
CREATE TABLE discoveries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    laureate_id INTEGER REFERENCES nobel_laureates(id) ON DELETE CASCADE,
    discovery TEXT NOT NULL,
    year_discovery INTEGER NOT NULL,
    research_funding REAL NOT NULL,
    citation_count INTEGER NOT NULL
);
""")

print("Tables created successfully.")

Tables created successfully.


## Data Insertion

Insert sample data into the newly created tables.

In [3]:
# Insert data into nobel_laureates table
nobel_laureates_data = [
    ('Marie Curie', 'Physics', 1903, 15000.00, 36),
    ('Dorothy Hodgkin', 'Chemistry', 1964, 273000.00, 54),
    ('Barbara McClintock', 'Physiology', 1983, 1900000.00, 81),
    ('Richard Feynman', 'Physics', 1965, 282000.00, 47),
    ('Frederick Sanger', 'Chemistry', 1958, 250000.00, 40),
    ('Werner Heisenberg', 'Physics', 1932, 159917.00, 31),
    ('Gertrude Elion', 'Physiology', 1988, 2100000.00, 70),
    ('Linus Pauling', 'Chemistry', 1954, 236000.00, 53),
    ('Max Planck', 'Physics', 1918, 38840.00, 60),
    ('Elizabeth Blackburn', 'Physiology', 2009, 10000000.00, 61)
]
cursor.executemany("INSERT INTO nobel_laureates (name, field, year_award, prize_amount, age_at_award) VALUES (?, ?, ?, ?, ?)", nobel_laureates_data)

# Insert data into discoveries table
discoveries_data = [
    (1, 'Discovery of Radium and Polonium', 1898, 5000.00, 12500),
    (2, 'Structure of Vitamin B12', 1954, 30000.00, 8900),
    (3, 'Mobile Genetic Elements', 1950, 35000.00, 7800),
    (4, 'Quantum Electrodynamics', 1948, 25000.00, 13400),
    (5, 'Protein and Insulin Sequencing', 1951, 28000.00, 9500),
    (6, 'Uncertainty Principle', 1927, 8000.00, 9800),
    (7, 'Drug Development Through Rational Design', 1980, 150000.00, 9200),
    (8, 'Chemical Bond Nature', 1931, 15000.00, 11200),
    (9, 'Quantum Theory', 1900, 3000.00, 14000),
    (10, 'Telomeres and Telomerase', 1984, 180000.00, 10500)
]
cursor.executemany("INSERT INTO discoveries (laureate_id, discovery, year_discovery, research_funding, citation_count) VALUES (?, ?, ?, ?, ?)", discoveries_data)

# Commit changes
conn.commit()

print("Data inserted successfully.")

Data inserted successfully.


## Data Analysis Questions

Use the connected database (`conn`) and pandas (`pd.read_sql`) to answer the following questions. Write your SQL query as a string and then execute it using `pd.read_sql(your_query_string, conn)` in the code cells below each question.

### Question 1: List the names of laureates who were awarded the Nobel Prize after the age of 50.

In [4]:
# Write your query for Question 1 here
pd.read_sql('''
SELECT name
FROM nobel_laureates
WHERE age_at_award > 50
''', conn)

,name
0,Dorothy Hodgkin
1,Barbara McClintock
2,Gertrude Elion
3,Linus Pauling
4,Max Planck
5,Elizabeth Blackburn


### Question 2: Calculate the total prize money awarded to laureates in the field of Physiology.

In [5]:
# Write your query for Question 2 here
pd.read_sql('''
SELECT SUM(prize_amount) AS total_prize_amount
FROM nobel_laureates
WHERE field LIKE 'Physiology'
''', conn)

,total_prize_amount
0,14000000.0


### Question 3: What is the average citation count for discoveries made before 1950?

In [7]:
# Write your query for Question 3 here
pd.read_sql('''
SELECT AVG(citation_count) as average_citation_count
FROM discoveries
WHERE year_discovery < 1950
''', conn)

,average_citation_count
0,12180.0


### Question 4: Calculate the average age of laureates when they received their award, grouped by field.

In [9]:
# Write your query for Question 4 here
pd.read_sql('''
SELECT AVG(age_at_award) as average_age, field
FROM nobel_laureates
GROUP BY field
''', conn)

,average_age,field
0,49.000000,Chemistry
1,43.500000,Physics
2,70.666667,Physiology


### Question 5: List the top three most cited discoveries along with their citation counts.

In [10]:
# Write your query for Question 5 here
pd.read_sql('''
SELECT discovery
FROM discoveries
ORDER BY citation_count DESC
LIMIT 3
''', conn)

,discovery
0,Quantum Theory
1,Quantum Electrodynamics
2,Discovery of Radium and Polonium


### Question 6: Find all laureates who have more than one discovery listed in the database.

*Note: The sample data only has one discovery per laureate. Your query should still be written to handle cases where a laureate might have multiple discoveries.*

In [16]:
# Write your query for Question 6 here
pd.read_sql('''
SELECT n.name
FROM nobel_laureates n, discoveries d1, discoveries d2
WHERE d1.laureate_id = d2.laureate_id AND d1.discovery != d2.discovery
''', conn)

,name


### Question 7: Calculate the total research funding for discoveries made by laureates in each field.

In [19]:
# Write your query for Question 7 here
pd.read_sql('''
SELECT SUM(d.research_funding), n.field
FROM nobel_laureates n, discoveries d
WHERE n.id = d.laureate_id
GROUP BY n.field
''', conn)

,SUM(d.research_funding),field
0,73000.0,Chemistry
1,41000.0,Physics
2,365000.0,Physiology


### Question 8: Which laureate had the longest gap between their discovery and receiving their Nobel Prize?

In [24]:
# Write your query for Question 8 here
pd.read_sql('''
SELECT n.name
FROM nobel_laureates n, discoveries d
WHERE n.id = d.laureate_id
ORDER BY n.year_award - d.year_discovery DESC
LIMIT 1
''', conn)

,name
0,Barbara McClintock


### Question 9: Find all laureates whose names contain the string "in" (case-insensitive).

In [25]:
# Write your query for Question 9 here
pd.read_sql('''
SELECT name
FROM nobel_laureates
WHERE name LIKE '%in%' COLLATE NOCASE
''', conn)

,name
0,Dorothy Hodgkin
1,Barbara McClintock
2,Linus Pauling


### Question 10: Find all discoveries that had research funding over $10,000 and more than 1000 citations.

In [28]:
# Write your query for Question 10 here
pd.read_sql('''
SELECT discovery
FROM discoveries
WHERE research_funding > 10000 AND citation_count > 1000
''', conn)

,discovery
0,Structure of Vitamin B12
1,Mobile Genetic Elements
2,Quantum Electrodynamics
3,Protein and Insulin Sequencing
4,Drug Development Through Rational Design
5,Chemical Bond Nature
6,Telomeres and Telomerase


### Question 11: What percentage of all discoveries were funded with more than $20,000?

In [30]:
# Write your query for Question 11 here
funding = pd.read_sql('''
SELECT research_funding
FROM discoveries
''', conn)

len(funding[funding['research_funding'] > 20000].index) / len(funding.index) * 100

60.0

### Question 12: Rank all discoveries by citation count within each field (Physics, Chemistry, etc.).

*Note: Requires SQLite version 3.25.0 or higher for window functions.*

In [34]:
# Write your query for Question 12 here
pd.read_sql('''
SELECT SUM(d.citation_count), field
FROM nobel_laureates n, discoveries d
WHERE n.id = d.laureate_id
GROUP BY field
ORDER BY SUM(d.citation_count) DESC
''', conn)

,SUM(d.citation_count),field
0,49700,Physics
1,29600,Chemistry
2,27500,Physiology


## Cleanup

Close the database connection when you are finished.

In [35]:
# Close the connection
conn.close()
print("Database connection closed.")

Database connection closed.
